<a href="https://colab.research.google.com/github/tmvfb/generalSVR-generator/blob/main/generalSVR_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetune RuGPT with huggingface
Finetune RuGPT model with huggingface.   
Example for RuGPT3Small taken from [here](https://github.com/ai-forever/ru-gpts/blob/master/examples/RuGPT3FinetuneHF.ipynb). 

## Install env

In [ ]:
import pandas as pd
import numpy as np
import random

In [ ]:
%%bash
git clone https://github.com/huggingface/transformers
cd transformers
pip install .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/transformers
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for transformers: filename=transformers-4.27.0.dev0-py3-none-any.whl size=6464540 sha256=9cb73ce70fc8f9745b2a43fcbaaa926498c87bfbe11131910774c17c842a040d
  Stored in directory: /tmp/pip-ephem-wheel-cache-kjen5pyl/wheels/15/57/14/0d2873a0295966ca166ea9d9225761a50cce27e4d6b0341fcc
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.27.0.dev0
    Uninstalling transformers-4.27.0.dev0:
      Successfully uninstalled transformers-4.27.0.dev0


fatal: destination path 'transformers' already exists and is not an empty directory.


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!mkdir models/

mkdir: cannot create directory ‘models/’: File exists


## Download files and build train/validation samples

In [ ]:
data_path = 'https://www.dropbox.com/s/spcwx6hejebsb0m/result.json?dl=1'

In [ ]:
data = pd.read_json(data_path, encoding='utf-8')
data.head(10)

,0
0,"Дорогие подписчики и гости канала! Канал ""Ге..."
1,"Добрый вечер, дорогие подписчики и гости канал..."
2,"Здравствуйте, дорогие наши подписчики и гости ..."
3,"Здравствуйте, дорогие подписчики и гости канал..."
4,"Здравствуйте, дорогие подписчики и гости канал..."
5,"Здравствуйте, дорогие подписчики и гости канал..."
6,"Здравствуйте, дорогие наши подписчики и гости ..."
7,"Здравствуйте, дорогие наши подписчики и гости ..."
8,"Дорогие подписчики и гости канала! Завтра, во ..."
9,Дорогие подписчики и гости канала! Сегодня мат...


In [ ]:
random.seed(1234)
np.random.seed(1234)

In [ ]:
data.shape

(678, 1)

In [ ]:
val_ind = random.sample(range(data.shape[0]), 100)
train = [data.iloc[i][0] for i in range(len(data)) if i not in val_ind]
valid = [data.iloc[i][0] for i in range(len(data)) if i in val_ind]
# train = list(np.random.choice(data.iloc[:, 0], size=1100))
# valid = list(np.random.choice(data.iloc[:, 0], size=250))

In [ ]:
len(train), len(valid)

(578, 100)

In [ ]:
with open("train.txt", "w", encoding='utf-8') as file:
    file.write("\n".join(train))

In [ ]:
with open("valid.txt", "w", encoding='utf-8') as file:
    file.write("\n".join(valid))

## Train 
The following code downloads RuGPT model and tokenizer from huggingface and finetunes model for generating essays.

In [ ]:
!wget https://raw.githubusercontent.com/huggingface/transformers/main/examples/pytorch/language-modeling/run_clm.py

--2023-02-11 20:40:08--  https://raw.githubusercontent.com/huggingface/transformers/main/examples/pytorch/language-modeling/run_clm.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26853 (26K) [text/plain]
Saving to: ‘run_clm.py.3’

run_clm.py.3        100%[===================>]  26.22K  --.-KB/s    in 0s      

2023-02-11 20:40:08 (131 MB/s) - ‘run_clm.py.3’ saved [26853/26853]



In [ ]:
!python run_clm.py \
    --model_name_or_path sberbank-ai/rugpt3small_based_on_gpt2 \
    --train_file train.txt \
    --validation_file valid.txt \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --block_size 2048 \
    --dataset_config_name plain_text \
    --do_train \
    --do_eval \
    --output_dir models/essays

2023-02-11 20:40:11.999158: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-11 20:40:11.999310: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-11 20:40:11.999331: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
INFO:__main__:Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=F

## Evaluate model

In [ ]:
import torch

In [ ]:
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [ ]:
tok = GPT2Tokenizer.from_pretrained("models/essays")

In [ ]:
model = GPT2LMHeadModel.from_pretrained("models/essays")

In [ ]:
model.cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50264, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [ ]:
text = "Дорогие подписчики и гости канала! Владимир Путин съел яичницу на завтрак."
inpt = tok.encode(text, return_tensors="pt")

In [ ]:
out = model.generate(inpt.cuda(), 
                     max_length=100, 
                     repetition_penalty=5.0, 
                     do_sample=True, 
                     top_k=20, 
                     top_p=1, 
                     temperature=1)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
tok.decode(out[0], skip_special_tokens=True)

'Дорогие подписчики и гости канала! Владимир Путин съел яичницу на завтрак. Как сообщают очевидцы, президент не мог дождаться начала трапезы из-за сильной боли в голове или во время еды почувствовал запах жареного мяса у себя под носом (возможно он принимал душ по рекомендации врачей). Но это всего лишь предположения журналистов издания «Вести недели», которые говорят о том как много проблем с пищеварением возникает после принятия пищи со столь отвратительным перегаром.  В общем то вчера утром Путину стало плохо'